## Preprocesado de datos de población (INE)

Este notebook realiza el **primer análisis exploratorio y preprocesado de los datos de población** obtenidos del Instituto Nacional de Estadística (INE).

El objetivo es **limpiar, filtrar y transformar** los datos originales, que incluyen información por municipio, edad, sexo, nacionalidad y periodo, con el fin de obtener un conjunto de datos estructurado y adecuado para los análisis posteriores del proyecto.


- Instituto Nacional de Estadística (INE) – Población por municipios

In [1]:
import pandas as pd

### Carga de datos y exploración inicial

In [2]:
# Crear dataframe con datos ine
ine_raw = pd.read_csv('../data/raw/INE-poblacion-grupo-edad.csv', sep=';')

/tmp/ipykernel_54986/2371885041.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  ine_raw = pd.read_csv('../data/raw/INE-poblacion-grupo-edad.csv', sep=';')


> Al crear el *DataFrame* con los datos de población, **pandas muestra un `DtypeWarning`**, indicando que la **columna 7 (`Total`) contiene tipos mezclados** (valores numéricos y texto).
>
> Este aviso se debe a que la columna `Total` incluye valores numéricos con **separador de miles (`.`)**, por ejemplo `'49.128.297'`, que pandas no reconoce automáticamente como números y, por tanto, interpreta la columna como texto (`object`).

In [3]:
# Verificar valores no numéricos en la columna 'Total'
mask = pd.to_numeric(ine_raw['Total'], errors='coerce').isna()
ine_raw.loc[mask, 'Total'].value_counts()

Total
49.128.297    1
48.619.695    1
48.085.361    1
47.486.727    1
47.400.798    1
             ..
1.052.992     1
1.052.174     1
1.054.185     1
1.059.612     1
1.065.406     1
Name: count, Length: 715, dtype: int64

In [4]:
# Volver a cargar los datos especificando los parámetros correctos para miles y decimales
ine_raw = pd.read_csv(
    '../data/raw/INE-poblacion-grupo-edad.csv',
    sep=';',
    thousands='.',
    decimal=',',
    low_memory=False
)

#### Exploración del dataset

In [5]:
# Dimensión del dataset
ine_raw.shape

(8103150, 8)

In [6]:
# Tipos de datos
ine_raw.dtypes

Total Nacional     object
Provincias         object
Municipios         object
Sexo               object
Edad               object
Nacionalidad       object
Periodo             int64
Total             float64
dtype: object

In [7]:
# Valores nulos por columna
ine_raw.isna().sum()

Total Nacional        0
Provincias          990
Municipios        52470
Sexo                  0
Edad                  0
Nacionalidad          0
Periodo               0
Total               594
dtype: int64

In [8]:
# Valores únicos por columna
ine_raw.nunique()

Total Nacional        1
Provincias           52
Municipios         8132
Sexo                  3
Edad                 22
Nacionalidad          3
Periodo               5
Total             40974
dtype: int64

In [9]:
# Descripción estadística del dataset
ine_raw.describe(include='all')

,Total Nacional,Provincias,Municipios,Sexo,Edad,Nacionalidad,Periodo,Total
count,8103150,8102160,8050680,8103150,8103150,8103150,8.103150e+06,8.102556e+06
unique,1,52,8132,3,22,3,NaN,NaN
top,Total Nacional,09 Burgos,01001 Alegría-Dulantzi,Total,Todas las edades,Total,NaN,NaN
freq,8103150,368280,990,2701050,368325,2701050,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,2.023000e+03,7.130220e+02
std,NaN,NaN,NaN,NaN,NaN,NaN,1.414214e+00,6.547710e+04
min,NaN,NaN,NaN,NaN,NaN,NaN,2.021000e+03,0.000000e+00
25%,NaN,NaN,NaN,NaN,NaN,NaN,2.022000e+03,1.000000e+00
50%,NaN,NaN,NaN,NaN,NaN,NaN,2.023000e+03,6.000000e+00
75%,NaN,NaN,NaN,NaN,NaN,NaN,2.024000e+03,4.100000e+01


In [10]:
# Vista previa del dataframe
ine_raw.sample(5)

,Total Nacional,Provincias,Municipios,Sexo,Edad,Nacionalidad,Periodo,Total
5373275,Total Nacional,37 Salamanca,37046 Béjar,Hombres,De 65 a 69 años,Española,2025,484.0
4122731,Total Nacional,"26 Rioja, La",26091 Lumbreras de Cameros,Hombres,De 5 a 9 años,Extranjera,2024,0.0
707115,Total Nacional,06 Badajoz,06078 Malpartida de la Serena,Total,De 80 a 84 años,Total,2025,13.0
2191583,Total Nacional,"15 Coruña, A",15074 Rois,Mujeres,De 10 a 14 años,Española,2022,82.0
6608806,Total Nacional,44 Teruel,44100 Estercuel,Hombres,De 70 a 74 años,Total,2024,9.0


### Análisis exploratorio y preparación de variables

In [11]:
# Renombrar columnas
ine_clean = ine_raw.rename(columns={
    'Total Nacional': 'total_nacional',
    'Provincias': 'provincia',
    'Municipios': 'municipio',
    'Sexo': 'sexo',
    'Edad': 'grupo_edad',
    'Nacionalidad': 'nacionalidad',
    'Periodo': 'anyo',
    'Total': 'poblacion'
})
ine_clean.describe(include='all')

,total_nacional,provincia,municipio,sexo,grupo_edad,nacionalidad,anyo,poblacion
count,8103150,8102160,8050680,8103150,8103150,8103150,8.103150e+06,8.102556e+06
unique,1,52,8132,3,22,3,NaN,NaN
top,Total Nacional,09 Burgos,01001 Alegría-Dulantzi,Total,Todas las edades,Total,NaN,NaN
freq,8103150,368280,990,2701050,368325,2701050,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,2.023000e+03,7.130220e+02
std,NaN,NaN,NaN,NaN,NaN,NaN,1.414214e+00,6.547710e+04
min,NaN,NaN,NaN,NaN,NaN,NaN,2.021000e+03,0.000000e+00
25%,NaN,NaN,NaN,NaN,NaN,NaN,2.022000e+03,1.000000e+00
50%,NaN,NaN,NaN,NaN,NaN,NaN,2.023000e+03,6.000000e+00
75%,NaN,NaN,NaN,NaN,NaN,NaN,2.024000e+03,4.100000e+01


#### Variable `total_nacional`

La variable `total_nacional` presenta un único valor constante para todas las observaciones ("Total Nacional"), por lo que no aporta información adicional ni capacidad discriminatoria para el análisis.

Dado que el estudio se centra en el análisis de población a nivel municipal y su posterior integración con otras fuentes territoriales y sanitarias, esta variable se elimina del conjunto de datos.

In [12]:
# Quitar la columna 'total_nacional' ya que no aporta información relevante
ine_clean = ine_clean.drop(columns=['total_nacional'])
ine_clean.describe(include='all')

,provincia,municipio,sexo,grupo_edad,nacionalidad,anyo,poblacion
count,8102160,8050680,8103150,8103150,8103150,8.103150e+06,8.102556e+06
unique,52,8132,3,22,3,NaN,NaN
top,09 Burgos,01001 Alegría-Dulantzi,Total,Todas las edades,Total,NaN,NaN
freq,368280,990,2701050,368325,2701050,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,2.023000e+03,7.130220e+02
std,NaN,NaN,NaN,NaN,NaN,1.414214e+00,6.547710e+04
min,NaN,NaN,NaN,NaN,NaN,2.021000e+03,0.000000e+00
25%,NaN,NaN,NaN,NaN,NaN,2.022000e+03,1.000000e+00
50%,NaN,NaN,NaN,NaN,NaN,2.023000e+03,6.000000e+00
75%,NaN,NaN,NaN,NaN,NaN,2.024000e+03,4.100000e+01


#### Variable `provincia`

La variable `provincia` contiene información compuesta por el código numérico de la provincia y su nombre, ambos en una única cadena de texto. Para facilitar el filtrado territorial y la posterior integración con otras fuentes de datos, se procede a separar esta variable en dos nuevas columnas:

- `cod_provincia`: código numérico de la provincia.
- `nombre_provincia`: nombre de la provincia.

Previamente, se eliminan las filas con valores nulos en esta variable, ya que no pueden ser utilizadas en un análisis territorial ni en procesos de integración de datos.

Una vez separadas las variables y eliminada la columna original, el conjunto de datos se filtra para conservar únicamente las provincias pertenecientes a la Comunitat Valenciana (Alicante/Alacant, Castellón/Castelló y Valencia/València), en coherencia con los objetivos del proyecto.


In [13]:
# Eliminar filas con valores nulos en la columna 'provincia'
ine_clean = ine_clean.dropna(subset=['provincia']).reset_index(drop=True)
ine_clean.describe(include='all')

,provincia,municipio,sexo,grupo_edad,nacionalidad,anyo,poblacion
count,8102160,8050680,8102160,8102160,8102160,8.102160e+06,8.101566e+06
unique,52,8132,3,22,3,NaN,NaN
top,09 Burgos,01001 Alegría-Dulantzi,Total,Todas las edades,Total,NaN,NaN
freq,368280,990,2700720,368280,2700720,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,2.023000e+03,4.754061e+02
std,NaN,NaN,NaN,NaN,NaN,1.414214e+00,1.555400e+04
min,NaN,NaN,NaN,NaN,NaN,2.021000e+03,0.000000e+00
25%,NaN,NaN,NaN,NaN,NaN,2.022000e+03,1.000000e+00
50%,NaN,NaN,NaN,NaN,NaN,2.023000e+03,6.000000e+00
75%,NaN,NaN,NaN,NaN,NaN,2.024000e+03,4.100000e+01


In [14]:
# Separar la variable provincia en cod_provincia y nombre_provincia
ine_clean[['cod_provincia', 'nombre_provincia']] = ine_clean['provincia'].str.split(' ', n=1, expand=True)
ine_clean['cod_provincia'] = ine_clean['cod_provincia'].astype(int)
ine_clean.describe(include='all')

,provincia,municipio,sexo,grupo_edad,nacionalidad,anyo,poblacion,cod_provincia,nombre_provincia
count,8102160,8050680,8102160,8102160,8102160,8.102160e+06,8.101566e+06,8.102160e+06,8102160
unique,52,8132,3,22,3,NaN,NaN,NaN,52
top,09 Burgos,01001 Alegría-Dulantzi,Total,Todas las edades,Total,NaN,NaN,NaN,Burgos
freq,368280,990,2700720,368280,2700720,NaN,NaN,NaN,368280
mean,NaN,NaN,NaN,NaN,NaN,2.023000e+03,4.754061e+02,2.665872e+01,NaN
std,NaN,NaN,NaN,NaN,NaN,1.414214e+00,1.555400e+04,1.489402e+01,NaN
min,NaN,NaN,NaN,NaN,NaN,2.021000e+03,0.000000e+00,1.000000e+00,NaN
25%,NaN,NaN,NaN,NaN,NaN,2.022000e+03,1.000000e+00,1.300000e+01,NaN
50%,NaN,NaN,NaN,NaN,NaN,2.023000e+03,6.000000e+00,2.600000e+01,NaN
75%,NaN,NaN,NaN,NaN,NaN,2.024000e+03,4.100000e+01,4.100000e+01,NaN


In [15]:
# Quitar la columna provincia original
ine_clean = ine_clean.drop(columns=['provincia'])

In [16]:
ine_clean['cod_provincia'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52])

In [17]:
ine_clean['nombre_provincia'].unique()

array(['Araba/Álava', 'Albacete', 'Alicante/Alacant', 'Almería', 'Ávila',
       'Badajoz', 'Balears, Illes', 'Barcelona', 'Burgos', 'Cáceres',
       'Cádiz', 'Castellón/Castelló', 'Ciudad Real', 'Córdoba',
       'Coruña, A', 'Cuenca', 'Girona', 'Granada', 'Guadalajara',
       'Gipuzkoa', 'Huelva', 'Huesca', 'Jaén', 'León', 'Lleida',
       'Rioja, La', 'Lugo', 'Madrid', 'Málaga', 'Murcia', 'Navarra',
       'Ourense', 'Asturias', 'Palencia', 'Palmas, Las', 'Pontevedra',
       'Salamanca', 'Santa Cruz de Tenerife', 'Cantabria', 'Segovia',
       'Sevilla', 'Soria', 'Tarragona', 'Teruel', 'Toledo',
       'Valencia/València', 'Valladolid', 'Bizkaia', 'Zamora', 'Zaragoza',
       'Ceuta', 'Melilla'], dtype=object)

Debido a que nuestro estudio se centra en la Comunidad Valenciana, vamos a filtrar el dataset para quedarnos solo con las provincias de CV.

In [18]:
ine_clean = ine_clean[ine_clean['nombre_provincia'].isin(['Alicante/Alacant', 'Castellón/Castelló', 'Valencia/València'])].reset_index(drop=True)
ine_clean.describe(include='all')

,municipio,sexo,grupo_edad,nacionalidad,anyo,poblacion,cod_provincia,nombre_provincia
count,536580,539550,539550,539550,539550.000000,5.395500e+05,539550.000000,539550
unique,542,3,22,3,NaN,NaN,NaN,3
top,"03001 Atzúbia, l'",Total,Todas las edades,Total,NaN,NaN,NaN,Valencia/València
freq,990,179850,24525,179850,NaN,NaN,NaN,264330
mean,NaN,NaN,NaN,NaN,2023.000000,7.750663e+02,26.311927,NaN
std,NaN,NaN,NaN,NaN,1.414215,1.767557e+04,19.560368,NaN
min,NaN,NaN,NaN,NaN,2021.000000,0.000000e+00,3.000000,NaN
25%,NaN,NaN,NaN,NaN,2022.000000,3.000000e+00,3.000000,NaN
50%,NaN,NaN,NaN,NaN,2023.000000,2.000000e+01,12.000000,NaN
75%,NaN,NaN,NaN,NaN,2024.000000,1.160000e+02,46.000000,NaN


#### Variable `municipio`

La variable `municipio` también contiene información compuesta por el código numérico del municipio y su nombre, ambos en una única cadena de texto. Para facilitar el filtrado territorial y la posterior integración con otras fuentes de datos, se procede a separar esta variable en dos nuevas columnas:

- `cod_municipio`: código numérico del municipio.
- `nombre_municipio`: nombre del municipio.

Previamente, se eliminan las filas con valores nulos en esta variable, ya que no pueden ser utilizadas en un análisis territorial ni en procesos de integración de datos.

In [19]:
# Eliminar filas con valores nulos en la columna 'municipio'
ine_clean = ine_clean.dropna(subset=['municipio']).reset_index(drop=True)
ine_clean.describe(include='all')

,municipio,sexo,grupo_edad,nacionalidad,anyo,poblacion,cod_provincia,nombre_provincia
count,536580,536580,536580,536580,536580.000000,536580.000000,536580.000000,536580
unique,542,3,22,3,NaN,NaN,NaN,3
top,"03001 Atzúbia, l'",Total,Todas las edades,Total,NaN,NaN,NaN,Valencia/València
freq,990,178860,24390,178860,NaN,NaN,NaN,263340
mean,NaN,NaN,NaN,NaN,2023.000000,389.678169,26.345018,NaN
std,NaN,NaN,NaN,NaN,1.414215,4997.303637,19.560903,NaN
min,NaN,NaN,NaN,NaN,2021.000000,0.000000,3.000000,NaN
25%,NaN,NaN,NaN,NaN,2022.000000,3.000000,3.000000,NaN
50%,NaN,NaN,NaN,NaN,2023.000000,20.000000,12.000000,NaN
75%,NaN,NaN,NaN,NaN,2024.000000,112.000000,46.000000,NaN


In [20]:
# Separar la variable municipio en cod_municipio y nombre_municipio
ine_clean[['cod_municipio', 'nombre_municipio']] = (
    ine_clean['municipio']
    .str.split(' ', n=1, expand=True)
)
ine_clean['cod_municipio'] = ine_clean['cod_municipio'].astype(int)
ine_clean.describe(include='all')

,municipio,sexo,grupo_edad,nacionalidad,anyo,poblacion,cod_provincia,nombre_provincia,cod_municipio,nombre_municipio
count,536580,536580,536580,536580,536580.000000,536580.000000,536580.000000,536580,536580.000000,536580
unique,542,3,22,3,NaN,NaN,NaN,3,NaN,542
top,"03001 Atzúbia, l'",Total,Todas las edades,Total,NaN,NaN,NaN,Valencia/València,NaN,"Atzúbia, l'"
freq,990,178860,24390,178860,NaN,NaN,NaN,263340,NaN,990
mean,NaN,NaN,NaN,NaN,2023.000000,389.678169,26.345018,NaN,26459.627306,NaN
std,NaN,NaN,NaN,NaN,1.414215,4997.303637,19.560903,NaN,19586.070731,NaN
min,NaN,NaN,NaN,NaN,2021.000000,0.000000,3.000000,NaN,3001.000000,NaN
25%,NaN,NaN,NaN,NaN,2022.000000,3.000000,3.000000,NaN,3139.000000,NaN
50%,NaN,NaN,NaN,NaN,2023.000000,20.000000,12.000000,NaN,12139.500000,NaN
75%,NaN,NaN,NaN,NaN,2024.000000,112.000000,46.000000,NaN,46131.000000,NaN


In [21]:
# Quitar columna municipio original
ine_clean = ine_clean.drop(columns=['municipio'])

In [22]:
ine_clean[['cod_municipio', 'nombre_municipio']].sample(5)

,cod_municipio,nombre_municipio
35573,3036,Benillup
355522,46084,Càrcer
240567,12111,Tírig
28533,3029,Benigembla
303560,46031,Alginet


El conjunto de datos contiene 542 municipios distintos de la Comunidad Valenciana.

#### Variable `sexo`

La variable `sexo` presenta tres categorías: `Total`, `Hombres` y `Mujeres`.  
Dado que esta desagregación puede resultar de interés en análisis posteriores, se decide mantener la variable sin aplicar transformaciones adicionales en esta fase del preprocesado.

In [23]:
ine_clean['sexo'] = ine_clean['sexo'].astype('category')
ine_clean['sexo'].cat.categories

Index(['Hombres', 'Mujeres', 'Total'], dtype='object')

#### Variable `grupo_edad`

La variable `grupo_edad` presenta intervalos de edad quinquenales, un grupo agregado (`Todas las edades`) y un intervalo abierto para edades avanzadas (`100 y más años`). Esta categorización es coherente con la estructura estándar de los datos del INE y no presenta valores anómalos.

En esta fase del preprocesado se mantiene la categoría `Todas las edades`, ya que puede resultar útil para validaciones y análisis agregados posteriores. Asimismo, se define un orden lógico de las categorías de edad para facilitar la interpretación y la correcta representación gráfica en fases posteriores del análisis.

In [24]:
# Convertir la columna grupo_edad a tipo categoría
ine_clean['grupo_edad'] = ine_clean['grupo_edad'].astype('category')

# Definir el orden de las categorías para grupo_edad
orden_edades = [
    'Todas las edades',
    'De 0 a 4 años', 'De 5 a 9 años', 'De 10 a 14 años',
    'De 15 a 19 años', 'De 20 a 24 años', 'De 25 a 29 años',
    'De 30 a 34 años', 'De 35 a 39 años', 'De 40 a 44 años',
    'De 45 a 49 años', 'De 50 a 54 años', 'De 55 a 59 años',
    'De 60 a 64 años', 'De 65 a 69 años', 'De 70 a 74 años',
    'De 75 a 79 años', 'De 80 a 84 años', 'De 85 a 89 años',
    'De 90 a 94 años', 'De 95 a 99 años', '100 y más años'
]

ine_clean['grupo_edad'] = pd.Categorical(
    ine_clean['grupo_edad'],
    categories=orden_edades,
    ordered=True
)

ine_clean['grupo_edad'].cat.categories

Index(['Todas las edades', 'De 0 a 4 años', 'De 5 a 9 años', 'De 10 a 14 años',
       'De 15 a 19 años', 'De 20 a 24 años', 'De 25 a 29 años',
       'De 30 a 34 años', 'De 35 a 39 años', 'De 40 a 44 años',
       'De 45 a 49 años', 'De 50 a 54 años', 'De 55 a 59 años',
       'De 60 a 64 años', 'De 65 a 69 años', 'De 70 a 74 años',
       'De 75 a 79 años', 'De 80 a 84 años', 'De 85 a 89 años',
       'De 90 a 94 años', 'De 95 a 99 años', '100 y más años'],
      dtype='object')

#### Variable `nacionalidad`

La variable `nacionalidad` presenta tres categorías: `Total`, `Española` y `Extranjera`. Esta clasificación es coherente con la estructura de los datos del INE y no presenta valores anómalos.

En esta fase del preprocesado se decide mantener la variable sin aplicar transformaciones adicionales, dado que su desagregación puede resultar de interés en análisis posteriores y no afecta negativamente a los procesos de integración de datos.

In [25]:
ine_clean['nacionalidad'] = ine_clean['nacionalidad'].astype('category')
ine_clean['nacionalidad'].cat.categories

Index(['Española', 'Extranjera', 'Total'], dtype='object')

#### Variable `anyo`

La variable `anyo` representa el periodo temporal de referencia de los datos de población. Tras analizar el rango de años disponibles en el conjunto de datos, se decide filtrar el dataset para conservar únicamente el año 2025.

Esta decisión se justifica porque las demás fuentes de datos utilizadas en el proyecto (delimitaciones territoriales y centros sanitarios) corresponden al mismo periodo temporal, lo que garantiza la coherencia temporal en la integración de las distintas fuentes y evita sesgos derivados de la comparación entre años distintos.

In [26]:
ine_clean['anyo'].unique()

array([2025, 2024, 2023, 2022, 2021])

In [27]:
ine_clean['anyo'] = ine_clean['anyo'].astype(int)
ine_clean = ine_clean[ine_clean['anyo'] == 2025].reset_index(drop=True)
ine_clean.describe(include='all')

,sexo,grupo_edad,nacionalidad,anyo,poblacion,cod_provincia,nombre_provincia,cod_municipio,nombre_municipio
count,107316,107316,107316,107316.0,107316.000000,107316.000000,107316,107316.000000,107316
unique,3,22,3,NaN,NaN,NaN,3,NaN,542
top,Hombres,Todas las edades,Española,NaN,NaN,NaN,Valencia/València,NaN,"Atzúbia, l'"
freq,35772,4878,35772,NaN,NaN,NaN,52668,NaN,198
mean,NaN,NaN,NaN,2025.0,404.426702,26.345018,NaN,26459.627306,NaN
std,NaN,NaN,NaN,0.0,5131.060505,19.560975,NaN,19586.143735,NaN
min,NaN,NaN,NaN,2025.0,0.000000,3.000000,NaN,3001.000000,NaN
25%,NaN,NaN,NaN,2025.0,3.000000,3.000000,NaN,3139.000000,NaN
50%,NaN,NaN,NaN,2025.0,21.000000,12.000000,NaN,12139.500000,NaN
75%,NaN,NaN,NaN,2025.0,119.000000,46.000000,NaN,46131.000000,NaN


#### Variable `poblacion`

La variable `poblacion` es una variable numérica que representa el número de habitantes para cada combinación de municipio, sexo, grupo de edad, nacionalidad y año. Tras el preprocesado inicial, la variable presenta un tipo de dato numérico adecuado y no contiene valores nulos.

En esta fase no se realiza ningún tratamiento adicional sobre la variable, ya que los valores extremos observados corresponden a municipios con mayor población o a agregaciones válidas de los datos, y no se consideran errores. Cualquier agregación o análisis específico sobre esta variable se realizará en la fase de análisis descriptivo.

In [28]:
ine_clean['poblacion'].dtype

dtype('float64')

In [29]:
ine_clean['poblacion'].isna().sum()

np.int64(0)

In [30]:
ine_clean['poblacion'].describe()

count    107316.000000
mean        404.426702
std        5131.060505
min           0.000000
25%           3.000000
50%          21.000000
75%         119.000000
max      841558.000000
Name: poblacion, dtype: float64

### Exportación del dataset de población

Una vez finalizado el proceso de limpieza, transformación y filtrado del conjunto de datos de población del INE, se procede a guardar el dataset resultante en la carpeta `data/processed`. Este conjunto de datos limpio y estructurado será utilizado posteriormente en la fase de integración con las demás fuentes de información del proyecto.

El archivo se almacena en formato CSV para facilitar su reutilización en futuros análisis o proyectos.

In [31]:
ine_clean.sample(5)

,sexo,grupo_edad,nacionalidad,anyo,poblacion,cod_provincia,nombre_provincia,cod_municipio,nombre_municipio
99015,Total,De 20 a 24 años,Total,2025,21.0,46,Valencia/València,46225,Sellent
81391,Total,De 15 a 19 años,Española,2025,256.0,46,Valencia/València,46136,Godelleta
14016,Mujeres,De 35 a 39 años,Total,2025,208.0,3,Alicante/Alacant,3071,Gata de Gorgos
88288,Mujeres,De 70 a 74 años,Española,2025,123.0,46,Valencia/València,46170,Moixent/Mogente
75877,Total,De 65 a 69 años,Española,2025,38.0,46,Valencia/València,46108,Chera


In [ ]:
ine_clean.to_csv('../data/processed/INE-poblacion-grupo-edad-clean.csv', index=False)